# Databases and Storage: Exercise Results


## 1. Database Connection and Table Listing
- Write Python code to establish a connection to a local SQLite or PostgreSQL database.
- After connecting, retrieve and display the names of all tables present in the database.
- This exercise will help you practice database connectivity and schema exploration using Python.


In [ ]:
# ## 1. Database Connection and Table Listing
# Connect to a local SQLite database and list all tables

import sqlite3

# Establish connection to SQLite database (creates 'test.db' if it doesn't exist)
conn = sqlite3.connect('test.db')
cursor = conn.cursor()

# Retrieve all table names in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print("Tables in the database:")
for table in tables:
    print(table[0])

cursor.close()
conn.close()


## 2. CRUD Operations

- Practice the four essential database operations: **Create**, **Read**, **Update**, and **Delete** (CRUD).
- Write Python code that connects to a database table and demonstrates each operation:
    - Insert a new record into the table.
    - Retrieve and display records based on a condition.
    - Update an existing record.
    - Delete a record from the table.
- Use parameterized queries for safety and ensure each step prints out the results to verify correctness.


In [ ]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('test.db')
cursor = conn.cursor()

# Create table (if not exists)
cursor.execute("CREATE TABLE IF NOT EXISTS users (id INTEGER PRIMARY KEY AUTOINCREMENT, name TEXT)")

# 1. Insert a new record
cursor.execute("INSERT INTO users (name) VALUES (?)", ('Alice',))
conn.commit()
print("After Insert:")
cursor.execute("SELECT * FROM users")
print(cursor.fetchall())

# 2. Retrieve and display records based on a condition
cursor.execute("SELECT * FROM users WHERE name = ?", ('Alice',))
print("Select where name='Alice':")
print(cursor.fetchall())

# 3. Update an existing record
cursor.execute("UPDATE users SET name = ? WHERE name = ?", ('Bob', 'Alice'))
conn.commit()
print("After Update:")
cursor.execute("SELECT * FROM users")
print(cursor.fetchall())

# 4. Delete a record
cursor.execute("DELETE FROM users WHERE name = ?", ('Bob',))
conn.commit()
print("After Delete:")
cursor.execute("SELECT * FROM users")
print(cursor.fetchall())

# Clean up
conn.close()



## 3. NoSQL Practice

- Practice working with NoSQL databases using Python.
- Insert a sample user document into MongoDB and retrieve it.
- Store a key-value pair in Redis and fetch the stored value.
- Explore the basic operations and data models of document and key-value stores.


In [ ]:
# NoSQL Practice: MongoDB (Document Store) & Redis (Key-Value Store)

# --- MongoDB: Insert and Retrieve a Document ---
from pymongo import MongoClient

# Connect to local MongoDB server
client = MongoClient('mongodb://localhost:27017/')
db = client['test']

# Insert a sample user document
user_doc = {'name': 'Alice', 'email': 'alice@example.com', 'age': 30}
db.users.insert_one(user_doc)

# Retrieve the inserted document
retrieved_user = db.users.find_one({'name': 'Alice'})
print("MongoDB document:", retrieved_user)

# --- Redis: Store and Retrieve a Key-Value Pair ---
import redis

# Connect to local Redis server
r = redis.Redis(host='localhost', port=6379, db=0)

# Store a key-value pair
r.set('user:1001', 'Alice')

# Retrieve the value
value = r.get('user:1001')
print("Redis value:", value.decode() if value else None)


## 4. Cloud Object Storage with AWS S3
- Use Python and boto3 to upload a local file to an AWS S3 bucket and then download it back to your machine.
- Demonstrate both the upload and download steps in your code.
- Use a test bucket and a sample file (e.g., 'local.txt'). Ensure your AWS credentials are configured.


In [ ]:
import boto3

# Initialize S3 client (ensure AWS credentials are configured)
s3 = boto3.client('s3')

# Define bucket and file names
bucket_name = 'your-test-bucket'  # replace with your test bucket name
local_file = 'local.txt'
s3_key = 'remote.txt'
downloaded_file = 'downloaded.txt'

# Upload local file to S3 bucket
s3.upload_file(local_file, bucket_name, s3_key)
print(f"Uploaded {local_file} to s3://{bucket_name}/{s3_key}")

# Download the file back from S3
s3.download_file(bucket_name, s3_key, downloaded_file)
print(f"Downloaded s3://{bucket_name}/{s3_key} to {downloaded_file}")


---

### Challenge
- Investigate the impact of adding an index to a large database table.
- Measure and compare the query execution times before and after creating an index on a frequently filtered or sorted column.
- Use the `EXPLAIN` command in SQL to analyze and compare the query plans with and without the index.
- Summarize your findings on how indexing affects query performance and execution strategy.


In [ ]:
import sqlite3
import time

# Setup: Create a large table and populate with data
conn = sqlite3.connect('test.db')
cursor = conn.cursor()
cursor.execute('DROP TABLE IF EXISTS sales')
cursor.execute('CREATE TABLE sales (id INTEGER PRIMARY KEY, customer TEXT, amount REAL, region TEXT)')
# Insert many rows
rows = [('Customer{}'.format(i % 1000), float(i % 100), 'Region{}'.format(i % 10)) for i in range(200_000)]
cursor.executemany('INSERT INTO sales (customer, amount, region) VALUES (?, ?, ?)', rows)
conn.commit()

# 1. Query execution time before index
query = "SELECT * FROM sales WHERE customer = 'Customer123'"
start = time.time()
cursor.execute(query)
result1 = cursor.fetchall()
time_no_index = time.time() - start

# 2. EXPLAIN plan before index
cursor.execute('EXPLAIN QUERY PLAN ' + query)
explain_no_index = cursor.fetchall()

# 3. Create index on customer column
cursor.execute('CREATE INDEX idx_customer ON sales(customer)')
conn.commit()

# 4. Query execution time after index
start = time.time()
cursor.execute(query)
result2 = cursor.fetchall()
time_with_index = time.time() - start

# 5. EXPLAIN plan after index
cursor.execute('EXPLAIN QUERY PLAN ' + query)
explain_with_index = cursor.fetchall()

print(f"Rows returned: {len(result1)}")
print(f"Time WITHOUT index: {time_no_index:.4f} seconds")
print("EXPLAIN plan WITHOUT index:", explain_no_index)
print(f"Time WITH index: {time_with_index:.4f} seconds")
print("EXPLAIN plan WITH index:", explain_with_index)

# 6. Summary
if time_with_index < time_no_index:
    print("\nSummary: Adding an index on the 'customer' column significantly reduced query execution time and changed the query plan to use the index, demonstrating the performance benefits of indexing for large tables.")
else:
    print("\nSummary: Indexing did not provide a measurable speedup for this query. This may be due to table size, data distribution, or database caching. In production, indexes typically accelerate queries on large tables for frequently filtered/sorted columns.")

cursor.close()
conn.close()
